In [20]:
#!pip install autogluon

In [21]:
import pandas as pd
from autogluon.tabular import TabularPredictor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection._split import _BaseKFold
import numpy as np



In [22]:
x_train_a = pd.read_csv('cleaned_data/A/x_train_a.csv')
y_train_a = pd.read_csv('cleaned_data/A/train_a.csv')
x_test_a = pd.read_csv('cleaned_data/A/x_test_a.csv')

In [23]:
x_train_b = pd.read_csv('cleaned_data/B/x_train_b.csv')
y_train_b = pd.read_csv('cleaned_data/B/train_b.csv')
x_test_b = pd.read_csv('cleaned_data/B/x_test_b.csv')

In [24]:
x_train_c = pd.read_csv('cleaned_data/C/x_train_c.csv')
y_train_c = pd.read_csv('cleaned_data/C/train_c.csv')
x_test_c = pd.read_csv('cleaned_data/C/x_test_c.csv')

In [25]:
class CustomTimeSeriesSplit(_BaseKFold):
    def __init__(self, n_splits, train_size=None, test_size=None):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.train_size = train_size
        self.test_size = test_size

    def split(self, X, y=None, groups=None):
        n_samples = len(X)
        indices = np.arange(n_samples)

        # Define initial sizes if not provided
        train_size = self.train_size or n_samples // (self.n_splits + 1)
        test_size = self.test_size or n_samples // self.n_splits

        for test_start in range(train_size + test_size, n_samples, test_size):
            train_end = test_start - test_size
            train_start = max(train_end - train_size, 0)
            yield indices[train_start:train_end], indices[train_end:test_start]

# Example usage
tscv = CustomTimeSeriesSplit(n_splits=5, train_size=720, test_size=120)

for train_idx, test_idx in tscv.split(x_train_a):
    print("Train indices:", train_idx)
    print("Test indices:", test_idx)

Train indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242

In [26]:
x_train_a_combined = x_train_a.merge(y_train_a, left_on='date_forecast', right_on='time', how='left')
x_train_a_combined['observed'] = x_train_a_combined['calc_year'].isna().astype(int)
train_data_a = x_train_a_combined.drop(['time', 'calc_year', 'calc_month', 'calc_day', 'calc_hour', 'date_forecast'], axis = 1)
x_test_a['observed'] = x_test_a['calc_year'].isna().astype(int)
test_data_a = x_test_a.drop([ 'calc_year', 'calc_month', 'calc_day', 'calc_hour'], axis = 1)

In [27]:
x_train_b_combined = x_train_b.merge(y_train_b, left_on='date_forecast', right_on='time', how='left')
x_train_b_combined['observed'] = x_train_b_combined['calc_year'].isna().astype(int)
train_data_b = x_train_b_combined.drop(['time', 'calc_year', 'calc_month', 'calc_day', 'calc_hour', 'date_forecast'], axis = 1)
x_test_b['observed'] = x_test_b['calc_year'].isna().astype(int)
test_data_b = x_test_b.drop([ 'calc_year', 'calc_month', 'calc_day', 'calc_hour'], axis = 1)

In [28]:
x_train_c_combined = x_train_c.merge(y_train_c, left_on='date_forecast', right_on='time', how='left')
x_train_c_combined['observed'] = x_train_c_combined['calc_year'].isna().astype(int)
train_data_c = x_train_c_combined.drop(['time', 'calc_year', 'calc_month', 'calc_day', 'calc_hour', 'date_forecast'], axis = 1)
x_test_c['observed'] = x_test_c['calc_year'].isna().astype(int)
test_data_c = x_test_c.drop([ 'calc_year', 'calc_month', 'calc_day', 'calc_hour'], axis = 1)

In [29]:
predictor_a= TabularPredictor(label ='pv_measurement',eval_metric= 'mean_absolute_error').fit(train_data = train_data_a, verbosity = 2,presets='best_quality', time_limit= 1500)

No path specified. Models will be saved in: "AutogluonModels\ag-20231027_203733\"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 1500s
AutoGluon will save models to "AutogluonModels\ag-20231027_203733\"
AutoGluon Version:  0.8.2
Python Version:     3.9.7
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Disk Space Avail:   712.83 GB / 1022.87 GB (69.7%)
Train Data Rows:    34022
Train Data Columns: 54
Label Column: pv_measurement
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (5733.42, 0.0, 631.53469, 1166.77607)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['b

In [30]:
predictor_b= TabularPredictor(label ='pv_measurement',eval_metric= 'mean_absolute_error').fit(train_data = train_data_b, verbosity = 2,presets='best_quality', time_limit= 1500)

No path specified. Models will be saved in: "AutogluonModels\ag-20231027_205954\"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 1500s
AutoGluon will save models to "AutogluonModels\ag-20231027_205954\"
AutoGluon Version:  0.8.2
Python Version:     3.9.7
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Disk Space Avail:   705.75 GB / 1022.87 GB (69.0%)
Train Data Rows:    27053
Train Data Columns: 54
Label Column: pv_measurement
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (1152.3, 0.0, 104.14301, 209.38441)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['bin

In [31]:
predictor_c= TabularPredictor(label ='pv_measurement',eval_metric= 'mean_absolute_error').fit(train_data = train_data_c, verbosity = 2,presets='best_quality', time_limit= 1500)

No path specified. Models will be saved in: "AutogluonModels\ag-20231027_212047\"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 1500s
AutoGluon will save models to "AutogluonModels\ag-20231027_212047\"
AutoGluon Version:  0.8.2
Python Version:     3.9.7
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Disk Space Avail:   701.49 GB / 1022.87 GB (68.6%)
Train Data Rows:    21684
Train Data Columns: 54
Label Column: pv_measurement
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and label-values can't be converted to int).
	Label info (max, min, mean, stddev): (999.6, 0.0, 93.42292, 177.80112)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['

In [40]:
predictor_a.leaderboard(silent=True)

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,-141.539701,181.143574,1234.655094,0.001000,0.593890,3,True,21
1,ExtraTreesMSE_BAG_L2,-143.513883,177.341799,959.951911,1.681380,8.754716,2,True,16
2,RandomForestMSE_BAG_L2,-144.038555,177.455744,1006.787970,1.795325,55.590775,2,True,14
3,XGBoost_BAG_L2,-144.321210,175.874443,958.893776,0.214025,7.696581,2,True,18
4,LightGBM_BAG_L2,-144.890581,175.856033,956.338390,0.195614,5.141195,2,True,13
5,NeuralNetFastAI_BAG_L2,-145.319000,176.632568,1022.942384,0.972149,71.745189,2,True,17
6,LightGBMLarge_BAG_L2,-145.701090,176.004544,964.638681,0.344125,13.441486,2,True,20
7,CatBoost_BAG_L2,-145.716589,175.720386,968.824353,0.059968,17.627158,2,True,15
8,WeightedEnsemble_L2,-146.249966,167.912739,202.984134,0.003999,0.959719,2,True,11
9,LightGBMXT_BAG_L1,-146.385025,93.444348,63.448674,93.444348,63.448674,1,True,3


In [39]:
predictor_b.leaderboard(silent=True)

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,-19.909853,210.444865,1135.660423,0.000000,0.555882,3,True,22
1,ExtraTreesMSE_BAG_L2,-20.069295,207.820702,955.268908,1.089397,5.198383,2,True,17
2,RandomForestMSE_BAG_L2,-20.251041,207.833977,982.985087,1.102672,32.914562,2,True,15
3,XGBoost_BAG_L2,-20.437782,206.899494,956.833282,0.168190,6.762757,2,True,19
4,NeuralNetFastAI_BAG_L2,-20.565154,207.599992,1004.499761,0.868687,54.429235,2,True,18
5,WeightedEnsemble_L2,-20.606708,134.011347,202.569824,0.002000,0.605179,2,True,12
6,LightGBM_BAG_L2,-20.668760,206.867877,953.945524,0.136573,3.874999,2,True,14
7,CatBoost_BAG_L2,-20.691030,206.775862,967.663532,0.044558,17.593007,2,True,16
8,LightGBMXT_BAG_L1,-20.692372,67.649413,59.696292,67.649413,59.696292,1,True,3
9,LightGBMLarge_BAG_L2,-20.740554,207.025917,962.491681,0.294612,12.421155,2,True,21


In [38]:
predictor_c.leaderboard(silent=True)

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,-18.095226,96.250997,1407.578288,0.001000,0.429702,3,True,17
1,ExtraTreesMSE_BAG_L2,-18.321350,94.244457,1267.255715,0.969157,3.223744,2,True,12
2,WeightedEnsemble_L2,-18.321961,91.167833,112.551134,0.002997,0.754423,2,True,7
3,LightGBMXT_BAG_L1,-18.334534,55.473234,55.493167,55.473234,55.493167,1,True,3
4,RandomForestMSE_BAG_L2,-18.502327,94.225471,1284.209791,0.950171,20.177820,2,True,10
5,XGBoost_BAG_L2,-18.563129,93.402879,1270.213686,0.127579,6.181714,2,True,14
6,NeuralNetFastAI_BAG_L2,-18.816467,93.807504,1304.667902,0.532204,40.635931,2,True,13
7,LightGBMLarge_BAG_L2,-18.837114,93.515321,1275.823694,0.240021,11.791723,2,True,16
8,LightGBM_BAG_L2,-18.879522,93.374313,1270.254750,0.099013,6.222779,2,True,9
9,CatBoost_BAG_L2,-18.920658,93.319304,1281.128314,0.044004,17.096343,2,True,11


In [41]:
nan_counts = x_train_a_combined.isna().sum()
print(nan_counts)

date_forecast                  0
absolute_humidity_2m:gm3      24
air_density_2m:kgm3           24
ceiling_height_agl:m        6124
clear_sky_energy_1h:J          0
                            ... 
month_sin                      0
month_cos                      0
time                           0
pv_measurement                 0
observed                       0
Length: 61, dtype: int64


In [42]:
nan_counts = x_train_b_combined.isna().sum()
print(nan_counts)

date_forecast                  0
absolute_humidity_2m:gm3      24
air_density_2m:kgm3           24
ceiling_height_agl:m        4249
clear_sky_energy_1h:J          0
                            ... 
month_sin                      0
month_cos                      0
time                           0
pv_measurement                 0
observed                       0
Length: 61, dtype: int64


In [43]:
nan_counts = x_train_c_combined.isna().sum()
print(nan_counts)

date_forecast                  0
absolute_humidity_2m:gm3      24
air_density_2m:kgm3           24
ceiling_height_agl:m        4457
clear_sky_energy_1h:J          0
                            ... 
month_sin                      0
month_cos                      0
time                           0
pv_measurement                 0
observed                       0
Length: 61, dtype: int64


In [46]:
predictor_a.feature_importance(train_)

These features in provided data are not utilized by the predictor and will be ignored: ['date_forecast', 'snow_drift:idx', 'calc_year', 'calc_month', 'calc_day', 'calc_hour', 'time']


Computing feature importance via permutation shuffling for 53 features using 5000 rows with 5 shuffle sets...
	4041.91s	= Expected runtime (808.38s per shuffle set)
	2829.77s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
direct_rad:W,195.379250,3.684251,1.516557e-08,5,202.965174,187.793325
clear_sky_rad:W,188.159996,4.189231,2.946608e-08,5,196.785680,179.534311
diffuse_rad:W,140.312012,3.552708,4.927980e-08,5,147.627088,132.996936
clear_sky_energy_1h:J,87.010289,2.923835,1.527764e-07,5,93.030504,80.990073
direct_rad_1h:J,61.078100,1.586406,5.456395e-08,5,64.344532,57.811669
sun_elevation:d,57.662960,2.282422,2.939483e-07,5,62.362500,52.963421
hour_sin,52.412805,1.105924,2.377250e-08,5,54.689918,50.135693
effective_cloud_cover:p,39.889815,1.630024,3.338432e-07,5,43.246056,36.533573
total_cloud_cover:p,32.765600,0.950129,8.475251e-08,5,34.721930,30.809271
ceiling_height_agl:m,29.285812,1.527638,8.852381e-07,5,32.431240,26.140384


In [48]:
predictor_b.feature_importance(train_data_a, time_limit = 200)

Computing feature importance via permutation shuffling for 54 features using 5000 rows with 10 shuffle sets... Time limit: 200s...


	18379.79s	= Expected runtime (1837.98s per shuffle set)


KeyboardInterrupt: 

In [33]:
y_pred_a = predictor_a.predict(test_data_a)
y_pred_a=pd.DataFrame(y_pred_a,columns=['pv_measurement'])
y_pred_a #print the DataFrame

,pv_measurement
0,0.292309
1,0.330604
2,0.566708
3,61.216633
4,359.298859
...,...
715,206.164001
716,91.017349
717,6.604767
718,0.882854


In [34]:
y_pred_b = predictor_b.predict(test_data_b)
y_pred_b=pd.DataFrame(y_pred_b,columns=['pv_measurement'])
y_pred_b #print the DataFrame

,pv_measurement
0,-0.071208
1,0.004299
2,-0.085042
3,4.366973
4,52.177723
...,...
715,43.364571
716,15.639206
717,2.651379
718,-0.074147


In [35]:
y_pred_c = predictor_c.predict(test_data_c)
y_pred_c=pd.DataFrame(y_pred_c,columns=['pv_measurement'])
y_pred_c #print the DataFrame

,pv_measurement
0,0.015412
1,0.009630
2,0.015379
3,1.766769
4,12.922245
...,...
715,62.870125
716,42.385735
717,21.623631
718,2.292880


In [36]:
predictions = pd.concat([y_pred_a,y_pred_b, y_pred_c], ignore_index= True)
predictions.tail()

,pv_measurement
2155,62.870125
2156,42.385735
2157,21.623631
2158,2.292880
2159,0.278274


In [37]:
predictions_df = predictions.rename(columns={'pv_measurement': 'prediction'})

sample_submission = pd.read_csv('sample_submission.csv')

# Convert all negative predictions to 0
predictions_df.loc[predictions_df['prediction'] < 0, 'prediction'] = 0

# Join the 'id' column from sample_submission with the predictions
sample_submission['prediction'] = predictions_df['prediction']

# Save to CSV
sample_submission.to_csv('autogluon_prediction.csv', index=False)
predictions_df

,prediction
0,0.292309
1,0.330604
2,0.566708
3,61.216633
4,359.298859
...,...
2155,62.870125
2156,42.385735
2157,21.623631
2158,2.292880


In [ ]:
tscv = TimeSeriesSplit(n_splits=5)

performance_metrics = []

for train_idx, test_idx in tscv.split(x_train_a):
    x_train_fold = x_train_a.iloc[train_idx]
    y_train_fold = y_train_a.iloc[train_idx]
    x_valid_fold = x_train_a.iloc[test_idx]
    y_valid_fold = y_train_a.iloc[test_idx]

    # Merge the data on the date
    train_data_combined = x_train_fold.merge(y_train_fold, left_on='date_forecast', right_on='time', how='left')

    # Drop redundant columns
    train_data_combined.drop(columns=['time'], inplace=True)


    # Define the name of the target column
    label = 'pv_measurement'  # Replace with the name of your target column
    
    # Train the model
    predictor = TabularPredictor(label=label, eval_metric= 'mean_absolute_error').fit(train_data_combined, time_limit= 900)
    
    # Validate the model
    predictions = predictor.predict(x_valid_fold)
    
    # Evaluate the performance (you can use any metric of your choice)
    mae = abs(predictions - y_valid_fold['pv_measurement']).mean()

    performance_metrics.append(mae)



No path specified. Models will be saved in: "AutogluonModels\ag-20231026_173422\"
Beginning AutoGluon training ... Time limit = 900s
AutoGluon will save models to "AutogluonModels\ag-20231026_173422\"
AutoGluon Version:  0.8.2
Python Version:     3.9.7
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Disk Space Avail:   698.58 GB / 1022.87 GB (68.3%)
Train Data Rows:    5672
Train Data Columns: 54
Label Column: pv_measurement
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (5241.280000000001, 0.0, 573.93415, 1096.90156)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGe

	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 4 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
		Fitting DatetimeFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Useless Original Features (Count: 6): ['elevation:m', 'snow_drift:idx', 'calc_year', 'calc_month', 'calc_day', 'calc_hour']
		These features carry no predictive signal and should be manually investigated.
		This is typically a feature which has the same value for all rows.
		These features do not need to be present at inference time.
	Unused Original Features (Count: 1): ['cloud_base_agl:m']
		These features were not used to generate a

[1000]	valid_set's l1: 123.368
[2000]	valid_set's l1: 119.993
[3000]	valid_set's l1: 118.322
[4000]	valid_set's l1: 117.503
[5000]	valid_set's l1: 117.11
[6000]	valid_set's l1: 116.959
[7000]	valid_set's l1: 116.843
[8000]	valid_set's l1: 116.77
[9000]	valid_set's l1: 116.733
[10000]	valid_set's l1: 116.715


	-116.7114	 = Validation score   (-mean_absolute_error)
	7.33s	 = Training   runtime
	0.22s	 = Validation runtime
Fitting model: LightGBM ... Training model for up to 891.29s of the 891.29s of remaining time.
	-129.2873	 = Validation score   (-mean_absolute_error)
	1.24s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: RandomForestMSE ... Training model for up to 889.99s of the 889.99s of remaining time.


[1000]	valid_set's l1: 129.538


	-160.3094	 = Validation score   (-mean_absolute_error)
	3.54s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: CatBoost ... Training model for up to 886.35s of the 886.35s of remaining time.
		<method '_train' of '_catboost._CatBoost' objects> returned a result with an error set


: 

No path specified. Models will be saved in: "AutogluonModels\ag-20231026_153540\"


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 3>:3                                                                              │
│                                                                                                  │
│   1 # Begin with A                                                                               │
│   2                                                                                              │
│ ❱ 3 predictor = TabularPredictor(label='pv_measurement').fit(train_data=x_train, tuning_data     │
│   4 predictions = predictor.predict()  # Assuming x_test is your test data                       │
│   5                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'x_train' is not defined